In [1]:
import json
import numpy as np

with open('cloverleaf.pbf.json') as f:
    geojson = json.load(f)

polylines = [f['geometry']['coordinates'] for f in geojson['features'] if f['properties'].get('type') == 'lane_border']
print(f'#polylines: {len(polylines)}')

#polylines: 808


In [2]:
# filter out some polylines
# https://geojson.io/#data=data:text/x-url,https%3A%2F%2Fraw.githubusercontent.com%2Fcubao%2Findex%2Fmaster%2Fdocs%2Fnotebooks%2Fcloverleaf.pbf.json

big_bbox = [
    [120.68710518619417, 31.322672789303823],
    [120.68710518619417, 31.32093957974284],
    [120.68941884319116, 31.32093957974284],
    [120.68941884319116, 31.322672789303823],
    [120.68710518619417, 31.322672789303823],
]
small_bbox = [
    [120.68575783266118, 31.320826179826653],
    [120.68575783266118, 31.320254055318017],
    [120.68652496796722, 31.320254055318017],
    [120.68652496796722, 31.320826179826653],
    [120.68575783266118, 31.320826179826653],
]
bbox = small_bbox

from shapely.geometry import Polygon, LineString
polygon = Polygon(bbox)
polylines = [p for p in polylines if not polygon.disjoint(LineString(p))]
print(f'#polylines: {len(polylines)} (filter by bbox)')

#polylines: 60 (filter by bbox)


In [3]:
# convert to ENU
from polyline_ruler.tf import lla2enu
anchor = polylines[0][0]
polylines = [lla2enu(llas, anchor_lla=anchor).astype(np.float32) for llas in polylines]

In [4]:
from fast_crossing import FastCrossing

fc = FastCrossing()
for line in polylines:
    fc.add_polyline(line)
fc.finish()

In [5]:
intersections = fc.intersections(z_offset_range=[1e-3, np.inf])
print(len(intersections))
print(intersections[0]) # intersection point, (t, s), (polyline_idx, segment_idx), (polyline_idx, segment_idx), 

27
(array([-50.93553133, -11.73873725]), array([0.50657666, 0.76975261]), array([13,  0], dtype=int32), array([41,  0], dtype=int32))


In [6]:
poles = []
for inter in intersections:
    xy, (t, s), (poly1, seg1), (poly2, seg2) = inter
    xyz1 = fc.coordinates(poly1, seg1, t)
    xyz2 = fc.coordinates(poly2, seg2, s)
    poles.append([xyz1, xyz2])
print(f'#poles: {len(poles)}')
    

#poles: 27


In [7]:
import k3d
import numpy as np

plot = k3d.plot()
for polyline in polylines:
    plot += k3d.line(polyline)
for line in poles:
    plot += k3d.line(line, color=0xff0000)
    
plot.display()

Output()